# 🔮 Day 5 - Spell 1: KNN Classification with Magical Creatures

In [ ]:
knitr::opts_chunk$set(echo = TRUE, warning = FALSE, message = FALSE)Sys.setlocale("LC_CTYPE", "en_US.UTF-8")

## 🎯 Learning GoalsIn this spell, you'll learn to:- 🔍 Explore magical creature data with visualizations- 🤖 Build your first KNN (K-Nearest Neighbors) classification model- 📊 Evaluate model performance with accuracy and confusion matrices- ⚡ Find the optimal K value for best predictions- 🔮 Make predictions for new magical creatures---## 📚 Load Our Magical LibrariesFirst, let's load the libraries we need for our machine learning magic!

In [ ]:
# Load our magical librarieslibrary(tidymodels)  # For machine learning magiclibrary(tidyverse)   # For data manipulation and visualizationlibrary(kknn)        # For KNN classification

## 📖 Load the Magical Creature DatasetOda has been collecting data about various magical creatures she's encountered. Let's see what she's discovered!

In [ ]:
# Load the magical creature dataset# This dataset contains information about various magical creatures Oda has encounteredcreatures <- read_csv("../datasets/magical_creatures.csv")# Take a look at our magical friendshead(creatures)

---## 🔍 Part 1: Exploring the Magical Creature DataLet's get to know our magical creatures better!

In [ ]:
# Let's see what variables we have to work with# Each creature has: size, magic_power, friendliness_score, and behavior (friendly/mischievous)glimpse(creatures)

In [ ]:
# How many creatures of each type do we have?creatures %>%  count(behavior)

### 🎨 Visualizing Our CreaturesLet's create a beautiful plot to see if we can spot any patterns!

In [ ]:
# Let's visualize our creatures!# Plot size vs magic_power, colored by behaviorggplot(creatures, aes(x = size, y = magic_power, color = behavior)) +  geom_point(size = 3, alpha = 0.7) +  labs(title = "Oda's Magical Creature Collection",       x = "Size",        y = "Magic Power",       color = "Behavior") +  theme_minimal()

### 🤔 Think About It!💡 **Question:** Can you see any patterns? Do friendly creatures tend to cluster together?---## ⚙️ Part 2: Setting Up Our KNN ModelNow let's prepare our data for machine learning magic!### 📊 Split the DataWe need to split our data so we can train our model and then validate how well it works on "new" creatures.

In [ ]:
# Split our data into training and validation sets# We'll use 75% for training, 25% for validation (to choose best K)set.seed(123)  # For reproducible resultscreature_split <- initial_split(creatures, prop = 0.75, strata = behavior)creature_train <- training(creature_split)creature_validation <- testing(creature_split)

In [ ]:
# Let's check our split worked wellcat("Training data:\n")creature_train %>% count(behavior)cat("\nValidation data:\n")creature_validation %>% count(behavior)

### 🍳 Create a Recipe for Data PreprocessingIn machine learning, we often need to "cook" our data before feeding it to the model. Here's our recipe!

In [ ]:
# Create a recipe for preprocessing our data# We'll standardize size and magic_power to make sure they're on the same scalecreature_recipe <- recipe(behavior ~ size + magic_power, data = creature_train) %>%  step_scale(all_predictors()) %>%     # Scale to standard deviation 1  step_center(all_predictors())        # Center around 0print("Our magical recipe is ready!")

### 🤖 Create Our KNN ModelNow let's create our KNN model. We'll start with K=5 neighbors.

In [ ]:
# Create our KNN model specification# We'll start with K=5 neighborsknn_model <- nearest_neighbor(neighbors = 5) %>%  set_engine("kknn") %>%  set_mode("classification")print("KNN model created with K=5 neighbors!")

---## 🎓 Part 3: Training Our KNN ClassifierTime to train our magical creature classifier!

In [ ]:
# Create a workflow that combines our recipe and modelcreature_workflow <- workflow() %>%  add_recipe(creature_recipe) %>%  add_model(knn_model)print("Workflow assembled - ready for training!")

In [ ]:
# Train our model!creature_fit <- creature_workflow %>%  fit(data = creature_train)print("🎉 Model training complete!")

In [ ]:
# Let's make predictions on our validation datacreature_predictions <- predict(creature_fit, creature_validation) %>%  bind_cols(creature_validation)# Look at our predictionshead(creature_predictions)

---## 📈 Part 4: Evaluating Our Magical PredictionsHow well did our model do? Let's find out!### 🎯 Calculate Accuracy

In [ ]:
# Calculate the accuracy of our predictions# Convert to factors for proper classification metricscreature_predictions_factor <- creature_predictions %>%  mutate(    behavior = as.factor(behavior),    .pred_class = as.factor(.pred_class)  )accuracy_result <- creature_predictions_factor %>%  accuracy(truth = behavior, estimate = .pred_class)print(paste("Our KNN model accuracy:", round(accuracy_result$.estimate, 3)))

### 📊 Confusion MatrixA confusion matrix shows us exactly where our model made mistakes.

In [ ]:
# Create a confusion matrix to see where we made mistakescreature_predictions_factor %>%  conf_mat(truth = behavior, estimate = .pred_class)

### 🎨 Visualize Predictions vs Reality

In [ ]:
# Visualize our predictions vs actual behaviorggplot(creature_predictions, aes(x = size, y = magic_power)) +  geom_point(aes(color = behavior, shape = .pred_class), size = 3, alpha = 0.8) +  labs(title = "Actual vs Predicted Creature Behavior",       subtitle = "Color = Actual, Shape = Predicted",       x = "Size",        y = "Magic Power",       color = "Actual Behavior",       shape = "Predicted Behavior") +  theme_minimal()

---## 🔧 Part 5: Testing Different K ValuesDifferent K values can give us different results. Let's find the best one!

In [ ]:
# Let's try different K values to see which works best!test_k_values <- function(k_val) {  knn_model_k <- nearest_neighbor(neighbors = k_val) %>%    set_engine("kknn") %>%    set_mode("classification")    workflow_k <- workflow() %>%    add_recipe(creature_recipe) %>%    add_model(knn_model_k)    fit_k <- workflow_k %>% fit(data = creature_train)    predictions_k <- predict(fit_k, creature_validation) %>%    bind_cols(creature_validation)    accuracy_k <- predictions_k %>%    mutate(      behavior = as.factor(behavior),      .pred_class = as.factor(.pred_class)    ) %>%    accuracy(truth = behavior, estimate = .pred_class) %>%    pull(.estimate)    return(accuracy_k)}

In [ ]:
# Test different K valuesk_values <- c(1, 3, 5, 7, 10)accuracies <- map_dbl(k_values, test_k_values)# Create a data frame with resultsk_results <- tibble(K = k_values, Accuracy = accuracies)print("Results for different K values:")print(k_results)

In [ ]:
# Plot K vs Accuracyggplot(k_results, aes(x = K, y = Accuracy)) +  geom_line(color = "orange", linewidth = 1) +  geom_point(color = "#73bbda", size = 3) +  labs(title = "KNN Performance: How K Affects Accuracy",       x = "Number of Neighbors (K)",       y = "Accuracy") +  theme_minimal()

---## 🔮 Part 6: Predict New Magical Creatures!Oda has discovered some new creatures! Can you predict their behavior?

In [ ]:
# Oda has discovered some new creatures! Can you predict their behavior?new_creatures <- tibble(  name = c("Sparkle Dragon", "Tiny Pixie", "Giant Troll"),  size = c(8.5, 1.2, 9.8),  magic_power = c(9.1, 7.8, 3.2))print("New creatures discovered:")print(new_creatures)

In [ ]:
# Make predictions for the new creatures# Use the best K value from your analysis abovebest_k <- k_results %>%  filter(Accuracy == max(Accuracy)) %>%  pull(K) %>%  first()print(paste("Best K value:", best_k))# Retrain model with best Kbest_knn_model <- nearest_neighbor(neighbors = best_k) %>%  set_engine("kknn") %>%  set_mode("classification")best_workflow <- workflow() %>%  add_recipe(creature_recipe) %>%  add_model(best_knn_model)best_fit <- best_workflow %>% fit(data = creature_train)# Predict behavior for new creaturesnew_predictions <- predict(best_fit, new_creatures) %>%  bind_cols(new_creatures)print("Predictions for new magical creatures:")print(new_predictions)

---## 🧪 Part 7: Final Model Testing with Independent Test SetNow let's test our final model on completely unseen data to get an unbiased performance estimate!

In [ ]:
# Load the independent test dataset# This data was never used for training or choosing Ktest_creatures <- read_csv("../datasets/magical_creatures_test.csv")print("Independent test dataset loaded:")print(paste("Number of test creatures:", nrow(test_creatures)))test_creatures %>% count(behavior)

In [ ]:
# Use our best model (with optimal K) to make predictions on test setfinal_test_predictions <- predict(best_fit, test_creatures) %>%  bind_cols(test_creatures)# Calculate final test accuracyfinal_test_accuracy <- final_test_predictions %>%  mutate(    behavior = as.factor(behavior),    .pred_class = as.factor(.pred_class)  ) %>%  accuracy(truth = behavior, estimate = .pred_class)print(paste("🎯 Final Test Accuracy (K =", best_k, "):", round(final_test_accuracy$.estimate, 3)))

In [ ]:
# Final confusion matrix on test setprint("🔍 Final Confusion Matrix on Test Set:")final_test_predictions %>%  mutate(    behavior = as.factor(behavior),    .pred_class = as.factor(.pred_class)  ) %>%  conf_mat(truth = behavior, estimate = .pred_class)

In [ ]:
# Visualize final test resultsggplot(final_test_predictions, aes(x = size, y = magic_power)) +  geom_point(aes(color = behavior, shape = .pred_class), size = 3, alpha = 0.8) +  labs(title = "Final Model Performance on Independent Test Set",       subtitle = paste("Best K =", best_k, "| Test Accuracy =", round(final_test_accuracy$.estimate, 3)),       x = "Size",        y = "Magic Power",       color = "True Behavior",       shape = "Predicted Behavior") +  theme_minimal()

### 🎓 Key Learning Points💡 **What we learned about model validation:**- **Training set**: Used to train the model- **Validation set**: Used to choose the best K value- **Test set**: Used for final, unbiased performance evaluation- This approach prevents overfitting and gives realistic performance estimates!---